<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/test_gcn_variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!python -c "import torch; print(torch.__version__)"


2.6.0+cu124


In [3]:
!python -c "import torch; print(torch.version.cuda)"



12.4


In [4]:
pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html



Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.3 MB/s eta 0:00:00


In [11]:
# Standard libraries
import numpy as np
from scipy import sparse
import seaborn as sns
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm
from IPython.display import Javascript  # Restrict height of output cell.

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
# import pyg_lib
import torch_sparse

# PyTorch geometric
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from torch_geometric import seed_everything

**STANDARD GCN pubmed**

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data

Using device: cuda


Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [8]:
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')

Number of nodes:          19717
Number of edges:          88648
Average node degree:      4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True


In [27]:
def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")


In [41]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=-1)

In [35]:
from torch_geometric.nn.models import GraphSAGE

model = GraphSAGE(
    in_channels=num_features,     # e.g., 500
    hidden_channels=64,        # e.g., 64
    out_channels=num_classes,          # e.g., 7 for Cora
    num_layers=2,
    dropout=0.5,                        # match your dropout
    act='relu',
    act_first=False,                   # matches default
    norm=None,                         # no normalization (optional)
    jk=None                            # no Jumping Knowledge
).to(device)


In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 16, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

print(model)

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)


In [19]:
def train(data, mask):
  model.train()
  optimizer.zero_grad()  # Clear gradients.
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  loss = criterion(out[mask], data.y[mask])  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  return loss

def test(data, mask):
  model.eval()
  out = model(data.x, data.edge_index)
  pred = out.argmax(dim=1)  # Use the class with highest probability.
  correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
  acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
  return acc

In [43]:
import time

#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
start_time = time.time()

for epoch in range(1, 101):
  loss = train(data, data.train_mask)
  train_acc = test(data, data.train_mask)
  val_acc = test(data, data.val_mask)
  print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}')

end_time = time.time()
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
print(f"Time taken: {end_time - start_time:.2f} seconds")

Epoch: 001, Train: 0.6667
Epoch: 002, Train: 0.7333
Epoch: 003, Train: 0.7667
Epoch: 004, Train: 0.6667
Epoch: 005, Train: 0.6500
Epoch: 006, Train: 0.6500
Epoch: 007, Train: 0.6833
Epoch: 008, Train: 0.7000
Epoch: 009, Train: 0.7167
Epoch: 010, Train: 0.7167
Epoch: 011, Train: 0.7167
Epoch: 012, Train: 0.7167
Epoch: 013, Train: 0.8167
Epoch: 014, Train: 0.8500
Epoch: 015, Train: 0.8500
Epoch: 016, Train: 0.8500
Epoch: 017, Train: 0.8500
Epoch: 018, Train: 0.8500
Epoch: 019, Train: 0.8500
Epoch: 020, Train: 0.8833
Epoch: 021, Train: 0.8667
Epoch: 022, Train: 0.8667
Epoch: 023, Train: 0.8667
Epoch: 024, Train: 0.8667
Epoch: 025, Train: 0.8500
Epoch: 026, Train: 0.8500
Epoch: 027, Train: 0.8500
Epoch: 028, Train: 0.8667
Epoch: 029, Train: 0.8833
Epoch: 030, Train: 0.9333
Epoch: 031, Train: 0.9333
Epoch: 032, Train: 0.9500
Epoch: 033, Train: 0.9500
Epoch: 034, Train: 0.9500
Epoch: 035, Train: 0.9500
Epoch: 036, Train: 0.9500
Epoch: 037, Train: 0.9500
Epoch: 038, Train: 0.9500
Epoch: 039, 

In [45]:
test_acc = test(data, data.test_mask)
test_acc

0.765

In [28]:
clean_gpu_memory()

Memory after cleanup: 55.94 MB
